A/B experiment analysis

In [1]:
import glob
from collections import namedtuple

import pandas as pd
import numpy as np
import scipy.stats as ss

import matplotlib.pyplot as plt

pd.set_option("display.precision", 3)

%matplotlib inline

In [2]:
experiment = "GRAPHBASED_VS_STICKY"

In [3]:
data = pd.concat([
    pd.read_json(data_path, lines=True)
    for data_path
    in glob.glob("../../data4exp/graphbased_vs_sticky_9e43/*/data.json")
])

data["treatment"] = data["experiments"].map(lambda experiments: experiments[experiment])

In [4]:
data

message               timestamp  user  track  time    latency  \
0        next 2025-05-04 09:42:32.348  5130  13121  1.00  8.299e-03   
1        next 2025-05-04 09:42:32.405  1143   1163  1.00  5.529e-02   
2        next 2025-05-04 09:42:32.406  5130  13117  0.72  2.940e-04   
3        next 2025-05-04 09:42:32.407  7007  28713  0.71  2.611e-04   
4        next 2025-05-04 09:42:32.407  5130  13122  0.53  2.320e-04   
...       ...                     ...   ...    ...   ...        ...   
16097    next 2025-05-04 09:48:29.017  6899  47381  0.33  2.413e-04   
16098    next 2025-05-04 09:48:29.060  4779  42893  1.00  4.081e-02   
16099    next 2025-05-04 09:48:29.146  4779  43543  0.64  4.092e-02   
16100    next 2025-05-04 09:48:29.237  4779  43543  0.00  4.324e-02   
16101    last 2025-05-04 09:48:29.283  4779  42890  0.26  5.460e-05   

       recommendation                     experiments treatment  
0             13117.0   {'GRAPHBASED_VS_STICKY': 'C'}         C  
1             10919.0  {'GRAPHBASED_VS_STICKY': 'T1'}        T1  
2             13122.0   {'GRAPHBASED_VS_STICKY': 'C'}         C  
3             29769.0   {'GRAPHBASED_VS_STICKY': 'C'}         C  
4             13118.0   {'GRAPHBASED_VS_STICKY': 'C'}         C  
...               ...                             ...       ...  
16097         37434.0   {'GRAPHBASED_VS_STICKY': 'C'}         C  
16098         42891.0  {'GRAPHBASED_VS_STICKY': 'T1'}        T1  
16099         42893.0  {'GRAPHBASED_VS_STICKY': 'T1'}        T1  
16100         12080.0  {'GRAPHBASED_VS_STICKY': 'T1'}        T1  
16101             NaN  {'GRAPHBASED_VS_STICKY': 'T1'}        T1  

[32204 rows x 9 columns]

In [5]:
data.groupby("treatment").count()

message  timestamp   user  track   time  latency  recommendation  \
treatment                                                                     
C            16105      16105  16105  16105  16105    16105           14086   
T1           16099      16099  16099  16099  16099    16099           14118   

           experiments  
treatment               
C                16105  
T1               16099

## Visualize sessions

In [6]:
Session = namedtuple("Session", ["timestamp", "tracks", "time", "latency"])

def sessionize(user_data):
    sessions = []
    session = None
    for _, row in user_data.sort_values("timestamp").iterrows():
        if session is None:
            session = Session(row["timestamp"], 0, 0, 0)
        
        session = session._replace(
            tracks = session.tracks + 1, 
            time = session.time + row["time"],
            latency = session.latency + row["latency"] * 1000,
        )
        
        if row["message"] == "last":
            sessions.append(session._asdict())
            session = None
    return sessions

In [7]:
sessions = (
    data
    .groupby(["user", "treatment"])
    .apply(sessionize)
    .explode()
    .apply(pd.Series)
)

/var/folders/w_/vyw93vzx5g910cxg3_srjr7r0000gn/T/ipykernel_25667/2717925121.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(sessionize)


In [8]:
figure, ax = plt.subplots(figsize=(15, 5))
for treatment, treatment_sessions in sessions.groupby("treatment"):
    reindexed_treatment_sessions = treatment_sessions.sort_values("timestamp").reset_index()
    ax.plot(reindexed_treatment_sessions.index, reindexed_treatment_sessions["tracks"], label=treatment)
ax.legend()
pass

In [9]:
figure, ax = plt.subplots(figsize=(15, 5))
for treatment, treatment_sessions in sessions.groupby("treatment"):
    reindexed_treatment_sessions = treatment_sessions.sort_values("timestamp").reset_index()
    ax.plot(reindexed_treatment_sessions.index, reindexed_treatment_sessions["time"], label=treatment)
ax.legend()
pass

## Analyze the experiment

In [10]:
user_level_data = (
  sessions
    .reset_index()
    .groupby(["user", "treatment"])
    .agg(
        {
            "timestamp": "count",
            "tracks": "sum",
            "time": "sum",
            "latency": "sum"
        }
    )
)

user_level_data["sessions"] = user_level_data["timestamp"]
user_level_data["mean_request_latency"] = user_level_data["latency"] / user_level_data["tracks"]
user_level_data["mean_tracks_per_session"] = user_level_data["tracks"] / user_level_data["sessions"]
user_level_data["mean_time_per_session"] = user_level_data["time"] / user_level_data["sessions"]

metrics = [
    "time", 
    "sessions", 
    "mean_request_latency", 
    "mean_tracks_per_session", 
    "mean_time_per_session"
]

user_level_metrics = user_level_data[metrics].copy().reset_index()

In [11]:
treatment_level_metrics = (
    user_level_metrics
    .groupby("treatment")[metrics]
    .agg(["count", "mean", "var"])
)

In [12]:
def dof(n_0, n_1, s2_0, s2_1):
    numerator = (s2_0 / n_0 + s2_1 / n_1) * (s2_0 / n_0 + s2_1 / n_1)
    denominator = s2_0 * s2_0 / n_0 / n_0 / (n_0 - 1) + s2_1 * s2_1 / n_1 / n_1 / (n_1 - 1)
    return numerator / denominator


def ci(n_0, n_1, s2_0, s2_1, alpha=0.05):
    return ss.t.ppf(1 - alpha/2, dof(n_0, n_1, s2_0, s2_1)) * np.sqrt(s2_0 / n_0 + s2_1 / n_1)


effects = []

control = [data for treatment, data in treatment_level_metrics.iterrows() if treatment == "C"][0]

for treatment, row in treatment_level_metrics.iterrows():
    if treatment == "C":
        continue
    
    for metric in metrics:
        control_mean = control[metric]["mean"]
        treatment_mean = row[metric]["mean"]
        
        effect = treatment_mean - control_mean
        conf_int = ci(
            control[metric]["count"],
            row[metric]["count"],
            control[metric]["var"],
            row[metric]["var"],
        )
        effects.append({
            "treatment": treatment,
            "metric": metric,
            "control_mean": control_mean,
            "treatment_mean": treatment_mean,
            "effect": effect / control_mean * 100,
            "lower": (effect - conf_int) / control_mean * 100,
            "upper": (effect + conf_int) / control_mean * 100,
            "significant": (effect + conf_int) * (effect - conf_int) > 0
        })

In [13]:
def color(value):
    return 'color:red;' if value < 0 else 'color:green;'

def background(value):
    return 'color:white;background-color:green' if value else 'color:white;background-color:red'
        

(
    pd.DataFrame(effects)[[
        "treatment", 
        "metric",
        "effect", 
        "upper", 
        "lower", 
        "control_mean", 
        "treatment_mean",
        "significant"
    ]]
    .sort_values(["metric", "treatment"], ascending=False)
    .style
    .applymap(color, subset=["effect", "upper", "lower"])
    .applymap(background, subset=["significant"])
)

/var/folders/w_/vyw93vzx5g910cxg3_srjr7r0000gn/T/ipykernel_25667/3627937693.py:21: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color, subset=["effect", "upper", "lower"])
/var/folders/w_/vyw93vzx5g910cxg3_srjr7r0000gn/T/ipykernel_25667/3627937693.py:22: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(background, subset=["significant"])
